In [1]:
from accutuning_client.client import Client

In [2]:
client = Client('localhost', 8000)

In [3]:
client.login('autoinsight', 'autoinsight')

True

In [4]:
# Experiment의 List를 불러온다.
experiments = client.experiments()
print(f'현재 Experiments는 총 {len(experiments)}개 있습니다.')
experiments

현재 Experiments는 총 1개 있습니다.


1 Experiments
Experiment(id=1, name=Experiment-1, dataset.name=diabetes2, dataset.colCount=11, status=finished, estimatorType=REGRESSOR, metric=NEG_MEAN_SQUARED_ERROR, modelsCnt=52)

In [5]:
# Local File에서 Experiment를 생성한다. 
experiment_new = client.create_experiment_from_file('/Users/ahaljh/Downloads/iris1.csv')
experiment_new

Experiment(id=2, name=Experiment-2, status=creating, estimatorType=classifier, metric=accuracy)

In [6]:
experiment_new

Experiment(id=2, name=Experiment-2, dataset.name=iris1, dataset.colCount=5, status=ready, estimatorType=CLASSIFIER, metric=ACCURACY)

In [7]:
# Run AutoML
experiment_new.run()

True

In [8]:
# Experiment의 List를 불러온다.
experiments = client.experiments()
print(f'현재 Experiments는 총 {len(experiments)}개 있습니다.')
experiments

현재 Experiments는 총 2개 있습니다.


2 Experiments
Experiment(id=2, name=Experiment-2, dataset.name=iris1, dataset.colCount=5, status=learning, estimatorType=CLASSIFIER, metric=ACCURACY, modelsCnt=4)
Experiment(id=1, name=Experiment-1, dataset.name=diabetes2, dataset.colCount=11, status=finished, estimatorType=REGRESSOR, metric=NEG_MEAN_SQUARED_ERROR, modelsCnt=52)

In [9]:
# 시간이 오래 걸리니까 완료된 experiment를 다시 선택
experiment = experiments.get(id=1)
# experiment = experiments[1]  # 이 형태도 가능함
experiment

Experiment(id=1, name=Experiment-1, dataset.name=diabetes2, dataset.colCount=11, status=finished, estimatorType=REGRESSOR, metric=NEG_MEAN_SQUARED_ERROR, modelsCnt=52)

In [10]:
# Leaderboard 정보 구해오기 
leaderboard = experiment.leaderboard()
print(f'leaderboad의 model 갯수는 {len(leaderboard)}')
leaderboard

leaderboad의 model 갯수는 52


[{'id': '22',
  'score': -2652.9879717803033,
  'trainScore': -2439.149837983871,
  'validScore': -2652.9879717803033,
  'testScore': 0.0,
  'estimatorName': 'random_forest',
  'generator': 'optuna-pycaret',
  'file': {'size': '185889', 'sizeHumanized': '185.9 kB'},
  'deployedStatus': 'DONE'},
 {'id': '24',
  'score': -2652.9879717803033,
  'trainScore': -2439.149837983871,
  'validScore': -2652.9879717803033,
  'testScore': 0.0,
  'estimatorName': 'random_forest',
  'generator': 'optuna-pycaret',
  'file': {'size': '185889', 'sizeHumanized': '185.9 kB'},
  'deployedStatus': None},
 {'id': '50',
  'score': -2658.6195136363635,
  'trainScore': -2575.376746935484,
  'validScore': -2658.6195136363635,
  'testScore': 0.0,
  'estimatorName': 'random_forest',
  'generator': 'optuna-pycaret',
  'file': {'size': '155297', 'sizeHumanized': '155.3 kB'},
  'deployedStatus': None},
 {'id': '51',
  'score': -2658.6195136363635,
  'trainScore': -2575.376746935484,
  'validScore': -2658.619513636363

In [11]:
# 가장 순위 높은 모델 선택
model = leaderboard.best_model()
# model = leaderboard[0]  # 이 형태도 가능함
model

{'id': '22',
 'score': -2652.9879717803033,
 'trainScore': -2439.149837983871,
 'validScore': -2652.9879717803033,
 'testScore': 0.0,
 'estimatorName': 'random_forest',
 'generator': 'optuna-pycaret',
 'file': {'size': '185889', 'sizeHumanized': '185.9 kB'},
 'deployedStatus': 'DONE'}

In [12]:
# 두 번째 순위 높은 모델 선택 (Best Model은 Run후 자동배포되므로..)
model = leaderboard[1]
model

{'id': '24',
 'score': -2652.9879717803033,
 'trainScore': -2439.149837983871,
 'validScore': -2652.9879717803033,
 'testScore': 0.0,
 'estimatorName': 'random_forest',
 'generator': 'optuna-pycaret',
 'file': {'size': '185889', 'sizeHumanized': '185.9 kB'},
 'deployedStatus': None}

In [13]:
# 모델 배포
model.deploy()

{'deployModel': {'deployment': {'id': '2', 'model': {'id': '24', '__typename': 'MlModelBaseType', 'deployedStatus': 'REQUEST'}}}}


In [15]:
# Deployment 정보 구해오기 
deployments = experiment.deployments()
print(f'배포된 모델은 {len(deployments)}개 입니다.')
deployments

배포된 모델은 2개 입니다.


[{'id': '2',
  'name': 'random_forest',
  'description': None,
  'status': 'DONE',
  'modelType': 'model',
  'modelPk': 24,
  'allMetricsJson': None,
  'createdAt': '2021-02-08T17:18:37.679374',
  'testScore': None,
  'model': {'id': '24',
   'trainScore': -2439.149837983871,
   'validScore': -2652.9879717803033},
  'file': {'url': 'http://localhost:8000/media/runtime_0001/runtimeprocess_0008/output/pipeline.pkl',
   'size': '183007',
   'sizeHumanized': '183.0 kB',
   'name': 'runtime_0001/runtimeprocess_0008/output/pipeline.pkl'}},
 {'id': '1',
  'name': 'random_forest',
  'description': None,
  'status': 'DONE',
  'modelType': 'model',
  'modelPk': 22,
  'allMetricsJson': None,
  'createdAt': '2021-02-08T17:17:58.534127',
  'testScore': None,
  'model': {'id': '22',
   'trainScore': -2439.149837983871,
   'validScore': -2652.9879717803033},
  'file': {'url': 'http://localhost:8000/media/runtime_0001/runtimeprocess_0007/output/pipeline.pkl',
   'size': '183007',
   'sizeHumanized': '

In [16]:
deployed_model = deployments[0]
deployed_model

{'id': '2',
 'name': 'random_forest',
 'description': None,
 'status': 'DONE',
 'modelType': 'model',
 'modelPk': 24,
 'allMetricsJson': None,
 'createdAt': '2021-02-08T17:18:37.679374',
 'testScore': None,
 'model': {'id': '24',
  'trainScore': -2439.149837983871,
  'validScore': -2652.9879717803033},
 'file': {'url': 'http://localhost:8000/media/runtime_0001/runtimeprocess_0008/output/pipeline.pkl',
  'size': '183007',
  'sizeHumanized': '183.0 kB',
  'name': 'runtime_0001/runtimeprocess_0008/output/pipeline.pkl'}}

In [17]:
# 모델 예측을 위해 Default값인 최빈값을 구함
columns = experiment.column_info()
most_frequent = columns.most_frequent_values()
most_frequent

{'age': '0.0162806757273067',
 'sex': '-0.044641636506989005',
 'bmi': '-0.0245287593917836',
 'bp': '-0.00567061055493425',
 's1': '-0.0373437341334407',
 's2': '0.0162224364339952',
 's3': '-0.0139477432193303',
 's4': '-0.0394933828740919',
 's5': '-0.0181182673078967',
 's6': '0.0030644094143683197'}

In [18]:
# 예측을 위한 input 값 생성 (그냥 최빈값 사용)
input_val = most_frequent
input_val 

{'age': '0.0162806757273067',
 'sex': '-0.044641636506989005',
 'bmi': '-0.0245287593917836',
 'bp': '-0.00567061055493425',
 's1': '-0.0373437341334407',
 's2': '0.0162224364339952',
 's3': '-0.0139477432193303',
 's4': '-0.0394933828740919',
 's5': '-0.0181182673078967',
 's6': '0.0030644094143683197'}

In [19]:
# 예측 
predict_val = deployed_model.predict(input_val)
predict_val

'112.18'